First, a crude method of detecting the green pixels of a plant.
Assumptions:
1. The plant is green (specifically, close to this arbitrarily chosen color: 44, 128, 31)
2. The plant fills 35% of the image frame

In [290]:
# Imports
from PIL import Image
import colorsys
import numpy as np

# Import the image
img = Image.open("image324x324inside.jpg")
# img = Image.open("image324x324outside.jpg")


# Define constants
START_COLOR = (44, 128, 31)         # A magic green color
MIN_SATURATION = 0.2                # Minimum saturation for a pixel to be considered
THRESHOLD = 80                      # A threshold, in euclidean distance, for color matching
PLANT_FRACTION = 0.30               # Fraction of the image filled by the plant
PLANT_FRACTION_TOLERANCE = 0.01     # Tolerance for the plant fraction
MAX_ITERATIONS = 20                 # Max iterations for search

threshold_squared = THRESHOLD * THRESHOLD

# Define a function that takes a comparison color and determines the fraction of pixels close to it
def compare_image_to_color(image, color):
    width, height = image.size
    pixels = image.load()
    count = 0

    for x in range(width):
        for y in range(height):
            r, g, b = pixels[x, y][:3]  # Ignore alpha channel if present
            dr = r - color[0]
            dg = g - color[1]
            db = b - color[2]
            distance_squared = dr * dr + dg * dg + db * db

            if distance_squared <= threshold_squared:
                count += 1

    return count / (width * height)

Now, to compensate for the overall brightness of the image, we should adjust the magic color value in value (HSV) until we get within our 35% metric.

In [291]:
match_color = colorsys.rgb_to_hsv(START_COLOR[0]/255, START_COLOR[1]/255, START_COLOR[2]/255)

match_fraction = -100.0

min_value = 0.0
curr_value = match_color[2]
max_value = 1.0

iter_count = 0

# Binary-ish search to the best value
while abs(match_fraction - PLANT_FRACTION) > PLANT_FRACTION_TOLERANCE and abs(max_value - min_value) > 0.005 and iter_count < MAX_ITERATIONS:
    iter_count += 1

    color_below = (curr_value - min_value) / 2 + min_value
    color_above = (max_value - curr_value) / 2 + curr_value

    rgb_below = colorsys.hsv_to_rgb(match_color[0], match_color[1], color_below)
    rgb_above = colorsys.hsv_to_rgb(match_color[0], match_color[1], color_above)

    frac_below = compare_image_to_color(img, (int(rgb_below[0]*255), int(rgb_below[1]*255), int(rgb_below[2]*255)))
    frac_above = compare_image_to_color(img, (int(rgb_above[0]*255), int(rgb_above[1]*255), int(rgb_above[2]*255)))

    if abs(frac_below - PLANT_FRACTION) < abs(frac_above - PLANT_FRACTION):
        # Closer by decreasing value
        max_value = curr_value
        curr_value = color_below
        match_fraction = frac_below
    else:
        # Closer by increasing value
        min_value = curr_value
        curr_value = color_above
        match_fraction = frac_above

    print(f"Iteration {iter_count}: Value {curr_value}, Fraction {match_fraction}")

match_color_rgb = colorsys.hsv_to_rgb(match_color[0], match_color[1], match_color[2])

print(f"Best match color: {int(match_color_rgb[0]*255)}, {int(match_color_rgb[1]*255)}, {int(match_color_rgb[2]*255)}")
print(f"Best match fraction: {match_fraction}")

Iteration 1: Value 0.25098039215686274, Fraction 0.31396700198140526
Iteration 2: Value 0.3764705882352941, Fraction 0.30918495656149975
Best match color: 44, 128, 31
Best match fraction: 0.30918495656149975


Next we make a matrix of locations meeting the conditions to be a "plant pixel". Adding a minimum saturation also helps to filter out any neutral pixels (black or white).

In [292]:
# Define a function that takes a comparison color and replaces the pixels close to it with red
def build_plant_pixel_matrix(image, color):
    width, height = image.size
    pixels = image.load()
    mat_out = np.zeros((height, width), dtype=bool)

    for x in range(width):
        for y in range(height):
            r, g, b = pixels[x, y][:3]  # Ignore alpha channel if present
            dr = r - color[0]
            dg = g - color[1]
            db = b - color[2]
            distance_squared = dr * dr + dg * dg + db * db

            if distance_squared <= threshold_squared and colorsys.rgb_to_hsv(r/255, g/255, b/255)[1] >= MIN_SATURATION:
                mat_out[x, y] = True

    return mat_out

# Create a copy of the original image to show the detected plant
plant_mat = build_plant_pixel_matrix(img, (int(match_color_rgb[0]*255), int(match_color_rgb[1]*255), int(match_color_rgb[2]*255)))

Now we need to find a way to discard points that are disconnected from the actual plant.

In [ ]:
plant_mat_second_pass = np.zeros((plant_mat.shape[0] // 3, plant_mat.shape[1] // 3), dtype=bool)

# TODO: Replace with some fancy method
plant_mat_second_pass = plant_mat.copy()

Now display the results and overlay the found pixels onto the original image.

In [294]:
# Show results
first_pass_img_array = (plant_mat.astype(np.uint8) * 255).transpose()
img_first_pass = Image.fromarray(first_pass_img_array)

second_pass_img_array = (plant_mat_second_pass.astype(np.uint8) * 255).transpose()
img_second_pass = Image.fromarray(second_pass_img_array)

overlay_img = img.copy()
for x in range(overlay_img.width):
    for y in range(overlay_img.height):
        if plant_mat_second_pass[x, y]:
            overlay_img.putpixel((x, y), (255, 0, 0))  # Mark detected plant pixels in red

img_first_pass.show()
img_second_pass.show()
overlay_img.show()